<a href="https://colab.research.google.com/github/imadjamil/coursera_capstone/blob/master/coursera_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coursera capstone project
In this notebook, I will be working on the applied data science capstone project.

## Import section

In [2]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import time
import pickle
#import ml_utils as mlu
#import config as cfg


print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"imjamil","key":"7a7fc7ce6b90ff210a11de696b6f8888"}'}

In [9]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [10]:
!chmod 600 ~/.kaggle/kaggle.json

In [11]:
!kaggle datasets list

ref                                                               title                                                 size  lastUpdated          downloadCount  
----------------------------------------------------------------  --------------------------------------------------  ------  -------------------  -------------  
christianlillelund/donald-trumps-rallies                          Donald Trump's Rallies                               720KB  2020-09-26 10:25:08            684  
heeraldedhia/groceries-dataset                                    Groceries dataset                                    257KB  2020-09-17 04:36:08           3235  
andrewmvd/trip-advisor-hotel-reviews                              Trip Advisor Hotel Reviews                             5MB  2020-09-30 08:31:20           1848  
balraj98/stanford-background-dataset                              Stanford Background Dataset                           17MB  2020-09-26 12:57:59            194  
nehaprabhavalkar/india

In [12]:
!kaggle datasets download -d sobhanmoosavi/us-accidents

 97% 225M/232M [00:02<00:00, 53.8MB/s]
100% 232M/232M [00:02<00:00, 83.3MB/s]


In [ ]:
!unzip us-accidents.zip -d /content/drive/My\ Drive/colab/training/coursera/9\_Capstone/data

Archive:  us-accidents.zip
  inflating: /content/drive/My Drive/colab/training/coursera/9_Capstone/data/US_Accidents_June20.csv  


## Data processing section

In [39]:
df = pd.read_csv(r'/content/drive/My Drive/colab/training/coursera/9_Capstone/data/US_Accidents_June20.csv')
df.shape

(3513617, 49)

In [ ]:
df.columns

In [40]:
df.drop(['ID', 'Source', 'Timezone', 'Airport_Code', 'Zipcode', 'Weather_Timestamp'],
        axis=1,
        inplace=True)

In [41]:
for col in df.columns:
  s = sum(df[col].isna())/df.shape[0]
  if s > 0.2:
    print('na in ' + col + ': ' + str(s))


na in TMC: 0.29451104090172603
na in End_Lat: 0.7054889590982739
na in End_Lng: 0.7054889590982739
na in Number: 0.6440269386219386
na in Wind_Chill(F): 0.5317167465890562
na in Wind_Speed(mph): 0.12938490450154358
na in Precipitation(in): 0.5765779252548016


In [42]:
df.drop(['TMC', 'End_Lat', 'End_Lng', 'Number', 'Wind_Chill(F)', 'Precipitation(in)'], 
        axis=1, 
        inplace=True)


In [43]:
for col in df.columns:
  s = sum(df[col].isna())/df.shape[0]
  if s > 0.01:
    print('na in ' + col + ': ' + str(s))

na in Temperature(F): 0.01870778744524517
na in Humidity(%): 0.01983340813754032
na in Pressure(in): 0.01590440847707647
na in Visibility(mi): 0.021589148731919274
na in Wind_Direction: 0.01675595262659533
na in Weather_Condition: 0.021669407906439432


In [44]:
df['Temperature(F)'].fillna(df['Temperature(F)'].mean(),inplace=True)
df['Humidity(%)'].fillna(df['Humidity(%)'].mean(),inplace=True)
df['Pressure(in)'].fillna(df['Pressure(in)'].mean(),inplace=True)
df['Visibility(mi)'].fillna(df['Visibility(mi)'].mean(),inplace=True)
df['Wind_Speed(mph)'].fillna(df['Wind_Speed(mph)'].mean(),inplace=True)

In [50]:
df.sort_values(['Start_Time', 'County', 'City'],inplace=True)
df['Weather_Condition'].fillna(method='ffill',inplace=True) 
df['Wind_Direction'].fillna(method='ffill',inplace=True)

In [51]:
for col in df.columns:
  s = sum(df[col].isna())/df.shape[0]
  if s > 0:
    print('na in ' + col + ': ' + str(s))

na in Description: 2.84607001844538e-07
na in City: 3.187598420658825e-05
na in Sunrise_Sunset: 3.2729805212121865e-05
na in Civil_Twilight: 3.2729805212121865e-05
na in Nautical_Twilight: 3.2729805212121865e-05
na in Astronomical_Twilight: 3.2729805212121865e-05


In [52]:
df.dropna(axis=0, 
          how='any', 
          subset=['Astronomical_Twilight', 'Nautical_Twilight', 'Civil_Twilight', 'Sunrise_Sunset', 'City', 'Description'],
          inplace=True)

In [56]:
for col in df.columns:
  s = sum(df[col].isna())/df.shape[0]
  if s > 0:
    print('na in ' + col + ': ' + str(s))

In [54]:
df.shape

(3513501, 37)

Index(['Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'Distance(mi)', 'Description', 'Street', 'Side', 'City', 'County',
       'State', 'Country', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way',
       'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset',
       'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'],
      dtype='object')